In [1]:
cd /home/muhammadbsheikh/workspace/try

/home/muhammadbsheikh/workspace/try


In [2]:
rep = '1'

In [3]:

def extract_features(model,dl):
    lbls = []
    model.eval()
    device = 'cuda:0'
    model.cuda(device)
    with torch.no_grad():
        features = None
        for batch in tqdm(dl, disable=True):
            
            images = batch[0]
            labels = batch[1]
            images = images.to(device)
            #labels = labels.to(device)

            output = model(images)
            lbls.append(labels)
            print(labels)

            if features is not None:
                features = torch.cat((features, output), 0)

            else:
                features = output        
            

    return features.cpu().numpy(),lbls


def flatten_list(t):
    flat_list = [item for sublist in t for item in sublist]
    flat_list = np.array(flat_list)
    return flat_list

In [4]:
# cnn2fft
# check PyTorch versions
# import standard PyTorch modules
import os
os.environ['CUDA_VISIBLE_DEVICES']="3"
import torch
import torch.nn as nn
import torch.nn.functional as F
import pretrainedmodels
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

# import torchvision module to handle image manipulation
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
from torch.autograd import Variable

# calculate train time, writing train data to files etc.
import time
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from IPython.display import clear_output


from efficientnet_pytorch import EfficientNet

from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import scipy
import scipy.fft

print(torch.__version__)
print(torchvision.__version__)

#device = torch.device("cuda")
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity
# torch.cuda.current_device()

# Helper class, help track loss, accuracy, epoch time, run time, 
# hyper-parameters etc. Also record to TensorBoard and write into csv, json

# import modules to build RunBuilder and RunManager helper classes
from collections  import OrderedDict
from collections import namedtuple
from itertools import product

# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
  @staticmethod
  def get_runs(params):

    Run = namedtuple('Run', params.keys())

    runs = []
    for v in product(*params.values()):
      runs.append(Run(*v))
    
    return runs

class RunManager():

  def __init__(self):

    # tracking every epoch count, loss, accuracy, time
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    
    self.epoch_val_loss = 0
    self.epoch_val_num_correct = 0
    self.epoch_start_time = None

    # tracking every run count, run data, hyper-params used, time
    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    # record model, loader and TensorBoard 
    self.network = None
    self.loader = None
    self.vloader = None
    self.tb = None

  # record the count, hyper-param, model, loader of each run
  # record sample images and network graph to TensorBoard  
  def begin_run(self, run, network, loader,vloader):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.vloader = vloader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.loader))
    images, labels = images.to(device), labels.to(device)
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(self.network, images)

  # when run ends, close TensorBoard, zero epoch count
  def end_run(self):
    self.tb.close()
    self.epoch_count = 0

  # zero epoch count, loss, accuracy, 
  def begin_epoch(self):
    self.epoch_start_time = time.time()

    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0

  # 
  def end_epoch(self):
    # calculate epoch duration and run duration(accumulate)
    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    # record epoch loss and accuracy
    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)
    
    vloss = self.epoch_val_loss / len(self.vloader.dataset)
    vaccuracy = self.epoch_val_num_correct / len(self.vloader.dataset)

    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('Loss', loss, self.epoch_count)
    self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
    
    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('vLoss', vloss, self.epoch_count)
    self.tb.add_scalar('vAccuracy', vaccuracy, self.epoch_count)

    # Record params to TensorBoard
    for name, param in self.network.named_parameters():
      self.tb.add_histogram(name, param, self.epoch_count)
      self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
    
    # Write into 'results' (OrderedDict) for all run related data
    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results["loss"] = loss
    results["val_loss"] = self.epoch_val_loss
    results["val_accuracy"] = self.epoch_val_num_correct
    results["accuracy"] = accuracy
    results["epoch duration"] = epoch_duration
    results["run duration"] = run_duration

    # Record hyper-params into 'results'
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)
    df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

    # display epoch information and show progress
    clear_output(wait=True)
    #display(df)

  def track_vloss_vacc(self, loss, acc):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_val_loss = loss.item()
    self.epoch_val_num_correct = acc
    
    
    # accumulate loss of batch into entire epoch loss
  def track_loss(self, loss):
    # multiply batch size so variety of batch sizes can be compared
    self.epoch_loss += loss.item() * self.loader.batch_size

  # accumulate number of corrects of batch into entire epoch num_correct
  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self._get_num_correct(preds, labels)

  @torch.no_grad()
  def _get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
  
  # save end results of all runs into csv, json for further a
  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, 
        orient = 'columns',
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
      json.dump(self.run_data, f, ensure_ascii=False, indent=4)

#set device
if torch.cuda.is_available():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_properties(device),torch.cuda.set_device(device),torch.cuda.current_device())

#Hyperparameters

#input_size
#num_classes = 50
#learning_rate = 0.00005
BATCH_SIZE = 32
epochs = 1

# put all hyper params into a OrderedDict, easily expandable
params = OrderedDict(
    exp='6',
    lr = [0.0005],#,0.001,0.0001,0.00001], # 0.001
    batch_size = [BATCH_SIZE], # 1000
    shuffle = [True] # True,False
    # Optimizer = [Adam,NAdam,RMSProp,Adamax,SGD,Adagrad,Adadelta]
)

TRAIN_DATA_PATH = "./datax/"+rep+"/train/"
TEST_DATA_PATH = "./datax/"+rep+"/test/"

transform = transforms.Compose([    
    transforms.Resize(299), # preffered size for network
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform)



train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True,  num_workers=4)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True,  num_workers=4)

m = RunManager()


#prepare model
model_name = 'inceptionresnetv2' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

model.last_linear = nn.Identity() #freeze the model
#num_ftrs = model.last_linear.in_features
#model.last_linear = nn.Linear(num_ftrs, 50)

for p in model.parameters():
    p.requires_grad =False
    
#num_ftrs = model.last_linear.in_features

    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
#model.fc = nn.Linear(num_ftrs, 50)

PATH = "models/IRv2.pt"

torch.save(model,PATH)
#model = torch.load(PATH)

optimizer = optim.Adam(model.parameters(), lr=0.0005)

def train(model,loader,epochs=60):
    model.to(device)
    model.train()   
    print('Training...')
    for epoch in range(epochs):
        start = time.time()
        model.train()
        m.begin_epoch()
        running_loss=0

        for i,batch in enumerate(loader,0):
                images = batch[0]
                labels = batch[1]
                images = images.to(device)
                labels = labels.to(device)
                
                preds = model(images)
                loss = F.cross_entropy(preds, labels) # Adam, SGD, RSPROP

                optimizer.zero_grad()
                loss = Variable(loss, requires_grad = True)
                loss.backward()
                optimizer.step()

                running_loss+=loss.data

                if i%10==9:
                    end=time.time()
                    #print ('[epoch %d,imgs %5d] time: %0.3f s'%(epoch+1,(i+1)*4,(end-start)))
                    print ('[epoch %d,imgs %5d] loss: %.7f  time: %0.3f s'%(epoch+1,(i+1)*4,running_loss/100,(end-start)))
                    #tb.add_scalar('Loss', loss, epoch+1)
                    start=time.time()
                    running_loss=0    
    

train(model,train_data_loader)


print('Extracting Features...')
feat,lbls = extract_features(model,train_data_loader)
# randomforest, logisticregression, SVM , KNN, LD,  


1.8.1
0.9.1
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28) None 0
Training...
[epoch 1,imgs    40] loss: 0.7381561  time: 1.593 s
[epoch 1,imgs    80] loss: 0.7364691  time: 0.922 s
[epoch 1,imgs   120] loss: 0.7424034  time: 0.923 s
[epoch 1,imgs   160] loss: 0.7409665  time: 0.927 s
[epoch 1,imgs   200] loss: 0.7348387  time: 0.935 s
[epoch 1,imgs   240] loss: 0.7349889  time: 0.923 s
[epoch 1,imgs   280] loss: 0.7381642  time: 0.924 s
[epoch 1,imgs   320] loss: 0.7404304  time: 0.919 s
[epoch 1,imgs   360] loss: 0.7364513  time: 0.918 s
[epoch 1,imgs   400] loss: 0.7401215  time: 0.918 s
[epoch 1,imgs   440] loss: 0.7385595  time: 0.920 s
[epoch 1,imgs   480] loss: 0.7406903  time: 0.922 s
[epoch 1,imgs   520] loss: 0.7373325  time: 0.922 s
[epoch 1,imgs   560] loss: 0.7337598  time: 0.921 s
[epoch 1,imgs   600] loss: 0.7374717  time: 0.922 s
[epoch 1,imgs   640] loss: 0.7406570  time: 0.923 s
[epoch 1,imgs   680] 

[epoch 6,imgs   240] loss: 0.7427058  time: 0.959 s
[epoch 6,imgs   280] loss: 0.7382180  time: 0.962 s
[epoch 6,imgs   320] loss: 0.7396926  time: 0.965 s
[epoch 6,imgs   360] loss: 0.7368746  time: 0.963 s
[epoch 6,imgs   400] loss: 0.7423671  time: 0.960 s
[epoch 6,imgs   440] loss: 0.7383612  time: 0.960 s
[epoch 6,imgs   480] loss: 0.7409030  time: 0.961 s
[epoch 6,imgs   520] loss: 0.7410789  time: 0.952 s
[epoch 6,imgs   560] loss: 0.7370366  time: 0.956 s
[epoch 6,imgs   600] loss: 0.7387341  time: 0.962 s
[epoch 6,imgs   640] loss: 0.7365826  time: 0.964 s
[epoch 6,imgs   680] loss: 0.7405027  time: 0.966 s
[epoch 6,imgs   720] loss: 0.7362674  time: 0.964 s
[epoch 6,imgs   760] loss: 0.7363520  time: 0.963 s
[epoch 6,imgs   800] loss: 0.7403989  time: 0.961 s
[epoch 6,imgs   840] loss: 0.7381018  time: 0.960 s
[epoch 6,imgs   880] loss: 0.7383973  time: 0.962 s
[epoch 6,imgs   920] loss: 0.7379249  time: 0.960 s
[epoch 6,imgs   960] loss: 0.7426349  time: 0.967 s
[epoch 6,img

[epoch 11,imgs   520] loss: 0.7391607  time: 0.959 s
[epoch 11,imgs   560] loss: 0.7387155  time: 0.956 s
[epoch 11,imgs   600] loss: 0.7394482  time: 0.957 s
[epoch 11,imgs   640] loss: 0.7372785  time: 0.958 s
[epoch 11,imgs   680] loss: 0.7372288  time: 0.958 s
[epoch 11,imgs   720] loss: 0.7386693  time: 0.967 s
[epoch 11,imgs   760] loss: 0.7373679  time: 0.957 s
[epoch 11,imgs   800] loss: 0.7412215  time: 0.963 s
[epoch 11,imgs   840] loss: 0.7405621  time: 0.966 s
[epoch 11,imgs   880] loss: 0.7365792  time: 0.965 s
[epoch 11,imgs   920] loss: 0.7388884  time: 0.961 s
[epoch 11,imgs   960] loss: 0.7410824  time: 0.958 s
[epoch 11,imgs  1000] loss: 0.7400840  time: 0.957 s
[epoch 11,imgs  1040] loss: 0.7399296  time: 0.958 s
[epoch 11,imgs  1080] loss: 0.7431235  time: 0.960 s
[epoch 11,imgs  1120] loss: 0.7386593  time: 0.960 s
[epoch 11,imgs  1160] loss: 0.7426770  time: 0.961 s
[epoch 11,imgs  1200] loss: 0.7393665  time: 0.960 s
[epoch 12,imgs    40] loss: 0.7406815  time: 1

[epoch 16,imgs   720] loss: 0.7407792  time: 0.956 s
[epoch 16,imgs   760] loss: 0.7387714  time: 0.956 s
[epoch 16,imgs   800] loss: 0.7405117  time: 0.957 s
[epoch 16,imgs   840] loss: 0.7371438  time: 0.956 s
[epoch 16,imgs   880] loss: 0.7347716  time: 0.955 s
[epoch 16,imgs   920] loss: 0.7373636  time: 0.957 s
[epoch 16,imgs   960] loss: 0.7389492  time: 0.957 s
[epoch 16,imgs  1000] loss: 0.7403759  time: 0.957 s
[epoch 16,imgs  1040] loss: 0.7394132  time: 0.957 s
[epoch 16,imgs  1080] loss: 0.7358475  time: 0.955 s
[epoch 16,imgs  1120] loss: 0.7397929  time: 0.957 s
[epoch 16,imgs  1160] loss: 0.7413535  time: 0.958 s
[epoch 16,imgs  1200] loss: 0.7364906  time: 0.957 s
[epoch 17,imgs    40] loss: 0.7379495  time: 1.370 s
[epoch 17,imgs    80] loss: 0.7357008  time: 0.965 s
[epoch 17,imgs   120] loss: 0.7391927  time: 0.965 s
[epoch 17,imgs   160] loss: 0.7412997  time: 0.963 s
[epoch 17,imgs   200] loss: 0.7385875  time: 0.969 s
[epoch 17,imgs   240] loss: 0.7416677  time: 0

[epoch 21,imgs   920] loss: 0.7381100  time: 0.956 s
[epoch 21,imgs   960] loss: 0.7391832  time: 0.957 s
[epoch 21,imgs  1000] loss: 0.7363527  time: 0.956 s
[epoch 21,imgs  1040] loss: 0.7396860  time: 0.956 s
[epoch 21,imgs  1080] loss: 0.7356641  time: 0.956 s
[epoch 21,imgs  1120] loss: 0.7372336  time: 0.961 s
[epoch 21,imgs  1160] loss: 0.7331828  time: 0.957 s
[epoch 21,imgs  1200] loss: 0.7390982  time: 0.959 s
[epoch 22,imgs    40] loss: 0.7395834  time: 1.316 s
[epoch 22,imgs    80] loss: 0.7420052  time: 0.956 s
[epoch 22,imgs   120] loss: 0.7402399  time: 0.958 s
[epoch 22,imgs   160] loss: 0.7419688  time: 0.958 s
[epoch 22,imgs   200] loss: 0.7424991  time: 0.956 s
[epoch 22,imgs   240] loss: 0.7418255  time: 0.956 s
[epoch 22,imgs   280] loss: 0.7414982  time: 0.955 s
[epoch 22,imgs   320] loss: 0.7381527  time: 0.955 s
[epoch 22,imgs   360] loss: 0.7385570  time: 0.955 s
[epoch 22,imgs   400] loss: 0.7389138  time: 0.955 s
[epoch 22,imgs   440] loss: 0.7403369  time: 0

...

[epoch 52,imgs   920] loss: 0.7394522  time: 0.959 s
[epoch 52,imgs   960] loss: 0.7394080  time: 0.958 s
[epoch 52,imgs  1000] loss: 0.7395017  time: 0.957 s
[epoch 52,imgs  1040] loss: 0.7361182  time: 0.958 s
[epoch 52,imgs  1080] loss: 0.7428353  time: 0.956 s
[epoch 52,imgs  1120] loss: 0.7391454  time: 0.957 s
[epoch 52,imgs  1160] loss: 0.7385240  time: 0.957 s
[epoch 52,imgs  1200] loss: 0.7397110  time: 0.959 s
[epoch 53,imgs    40] loss: 0.7374780  time: 1.345 s
[epoch 53,imgs    80] loss: 0.7417347  time: 0.958 s
[epoch 53,imgs   120] loss: 0.7413322  time: 0.957 s
[epoch 53,imgs   160] loss: 0.7353290  time: 0.957 s
[epoch 53,imgs   200] loss: 0.7378083  time: 0.957 s
[epoch 53,imgs   240] loss: 0.7361988  time: 0.956 s
[epoch 53,imgs   280] loss: 0.7360075  time: 0.956 s
[epoch 53,imgs   320] loss: 0.7407575  time: 0.960 s
[epoch 53,imgs   360] loss: 0.7435837  time: 0.957 s
[epoch 53,imgs   400] loss: 0.7416176  time: 0.954 s
[epoch 53,imgs   440] loss: 0.7370093  time: 0

[epoch 57,imgs  1120] loss: 0.7375628  time: 0.959 s
[epoch 57,imgs  1160] loss: 0.7412940  time: 0.960 s
[epoch 57,imgs  1200] loss: 0.7369132  time: 0.958 s
[epoch 58,imgs    40] loss: 0.7411276  time: 1.359 s
[epoch 58,imgs    80] loss: 0.7408227  time: 0.961 s
[epoch 58,imgs   120] loss: 0.7411598  time: 0.955 s
[epoch 58,imgs   160] loss: 0.7370255  time: 0.956 s
[epoch 58,imgs   200] loss: 0.7359678  time: 0.955 s
[epoch 58,imgs   240] loss: 0.7398127  time: 0.955 s
[epoch 58,imgs   280] loss: 0.7386993  time: 0.954 s
[epoch 58,imgs   320] loss: 0.7383450  time: 0.961 s
[epoch 58,imgs   360] loss: 0.7360585  time: 0.953 s
[epoch 58,imgs   400] loss: 0.7369435  time: 0.954 s
[epoch 58,imgs   440] loss: 0.7398252  time: 0.952 s
[epoch 58,imgs   480] loss: 0.7375010  time: 0.952 s
[epoch 58,imgs   520] loss: 0.7369009  time: 0.953 s
[epoch 58,imgs   560] loss: 0.7382792  time: 0.956 s
[epoch 58,imgs   600] loss: 0.7374447  time: 0.953 s
[epoch 58,imgs   640] loss: 0.7387606  time: 0

tensor([23,  7, 49, 28,  9, 37, 16,  0, 26,  2, 25, 21, 32, 11, 23,  3])
tensor([40, 41,  1, 19, 35, 32, 19, 25,  5, 42, 17, 18,  6, 12, 48, 39])
tensor([21, 44, 20, 27,  9, 44, 43, 21, 18, 17, 29,  3, 27, 33,  3,  7])
tensor([ 2, 40,  5,  7, 40,  6, 34, 14,  1, 14, 26, 36, 45, 27, 15, 18])
tensor([12, 41, 23, 25, 31,  1, 11, 27, 30, 30, 40, 45, 27, 45, 26,  7])
tensor([38,  6, 41,  7, 41, 13,  1,  7, 22, 49, 11, 29, 20,  7, 48, 16])
tensor([ 1, 16,  5, 45, 20, 17, 24, 13, 41, 48, 11,  1, 34, 12, 33, 45])
tensor([23,  5,  4, 41,  1, 23, 32, 10, 35, 25, 11, 49, 48, 35,  3, 12])
tensor([37, 14, 39, 12, 27, 47, 21, 15, 28, 24, 41,  0, 46,  4, 35, 25])
tensor([16, 11,  3, 20, 11, 35, 36, 39, 11, 23, 24, 47, 46, 12, 48, 32])
tensor([30,  9, 10,  0,  4,  7, 46, 25, 34, 25, 11,  2, 19, 41, 26, 41])
tensor([36, 11, 13, 36, 42, 47,  6, 11, 11, 24, 38, 41, 11, 12, 26, 28])
tensor([ 7, 25, 15, 34, 10, 25,  9, 37, 19,  7, 16, 17, 14, 39,  0, 38])
tensor([41, 47, 13, 49, 10, 19, 34,  3, 10,  3, 32,

tensor([14, 43, 14, 44, 16, 43, 34, 14,  2,  0,  6, 35, 28, 33,  9,  1])
tensor([14, 37, 39, 39,  2, 41, 25, 29, 22, 11, 23, 19, 15,  9, 41, 10])
tensor([ 3,  5, 44, 21, 35, 45, 46,  9, 20,  4, 12, 45, 30, 39, 38, 41])
tensor([23, 47,  7, 45, 46, 49, 23, 28, 21,  5,  2,  9, 14, 40,  2,  4])
tensor([43,  1,  0,  4, 46,  2,  4, 22, 25, 20, 44, 24, 26, 44, 29, 49])
tensor([49, 48, 44, 35, 23,  3,  7, 17, 10,  9, 36, 18, 30, 13, 10, 33])
tensor([39, 10, 43, 31, 19, 20, 19, 12, 48, 22, 10, 28, 21, 17,  5, 42])
tensor([38, 23, 46, 46, 38, 48, 10,  2, 12,  2, 49, 30, 45, 26,  0,  1])
tensor([19, 25, 22, 32, 33, 12, 35,  2, 30, 27, 46, 23, 25, 13, 19, 34])
tensor([23, 33, 39, 21, 46,  9,  3,  2, 36, 24,  2, 34, 35, 32, 40, 28])
tensor([39, 39, 41, 32, 29,  6,  2, 40, 43, 21, 46, 26, 17,  6, 41, 33])
tensor([37, 23, 34,  2, 46, 21, 24, 15,  3, 16, 37,  1, 40, 32, 46, 17])
tensor([19, 37, 12, 32, 34, 43, 28, 36,  0, 31, 45,  0, 33, 30, 41, 12])
tensor([33, 23, 31, 29, 31, 45, 43, 46, 34, 29, 16,

tensor([ 7, 14, 43,  6, 46, 43, 34,  9, 36,  9, 45, 27, 33, 11, 16, 41])
tensor([27, 14, 14, 32,  2,  2,  3, 27, 13, 41, 35, 43,  1, 15,  6, 38])
tensor([13, 22, 26, 13, 21,  3, 43, 49, 47, 36,  5, 47, 35, 30, 25, 24])
tensor([ 4, 34, 49,  6, 24, 10, 11, 28, 44, 23, 18, 20, 39, 27, 41,  9])
tensor([13, 24,  0, 29,  1, 14, 11,  5, 30,  7, 23, 27, 36, 13, 24, 43])
tensor([25,  0,  7, 12, 36, 47, 11, 39, 10,  0, 43, 12, 35, 45, 37, 42])
tensor([11, 46,  1, 41, 20, 16, 31, 43, 28, 34,  0,  2, 16, 21, 14, 42])
tensor([13, 26,  4, 24, 38, 11, 37, 39, 18,  7, 40,  6, 49, 37, 26, 45])
tensor([49, 19, 38,  9, 39,  4,  5, 45, 30, 19, 34, 20, 14, 12,  6, 39])
tensor([20, 12,  3, 12, 21, 36, 28,  6, 36, 20, 30, 34, 27, 25, 35, 18])
tensor([26, 32, 36,  4, 44, 32, 27, 19, 30, 25, 41, 29,  6, 18, 36, 40])
tensor([26,  4, 46,  6, 13, 11, 11, 49, 18, 47, 10, 14,  9, 16, 11,  9])
tensor([21, 16, 15, 27, 13, 11, 17, 39,  1, 21, 38, 48, 27, 16, 18, 19])
tensor([13, 21, 20, 16,  6, 25, 13, 20, 28,  5,  5,

In [5]:
feat,lbls = extract_features(model,train_data_loader)

tensor([43, 25, 13,  1, 30, 28, 24,  0, 31, 20,  2, 22, 13, 32, 18, 49])
tensor([ 2, 24, 45, 45, 44, 14, 34, 43, 11, 28, 27,  6, 22,  6, 43, 22])
tensor([49,  1,  6, 14, 41, 36, 44, 26, 32, 35, 49,  0, 23,  9, 48, 21])
tensor([26, 16,  3, 19, 33, 40, 39,  1, 25,  4, 18, 26, 45, 15, 41,  4])
tensor([ 6, 24,  6, 41, 11, 28, 20,  0,  0, 46, 43,  3, 11, 44, 46, 33])
tensor([ 2,  5, 14, 14, 43, 23, 36, 33,  7,  8, 13,  7, 42, 38, 33, 49])
tensor([ 6, 27, 34, 23, 28, 34,  8, 22,  5, 22, 22, 15, 28, 46, 28,  0])
tensor([43,  5,  2, 35,  3, 29, 12, 41, 36, 14, 34, 46, 40, 41, 14, 41])
tensor([29, 32,  6, 21, 15, 28, 21, 30,  3, 26, 32, 28,  8,  9, 21,  0])
tensor([21, 31, 16,  1, 48, 47, 17, 36, 43, 13,  6, 35, 33, 42, 26, 34])
tensor([25, 37, 26, 18,  0, 49, 25, 12, 38, 43, 13, 32, 18,  4,  5, 23])
tensor([45, 45, 36, 24, 22, 21, 26, 28, 26, 47, 23,  2, 18, 42,  9, 35])
tensor([32,  2, 16,  1, 11, 32, 19, 22,  1,  8, 17, 44, 26, 38, 23,  2])
tensor([ 5, 39, 45, 45, 41, 11,  7,  2,  4, 18, 21,

tensor([35, 10, 40,  1, 40, 19,  0,  2, 15, 42, 38, 45, 29, 11, 43, 11])
tensor([26, 36, 41, 41,  7, 30, 11, 31,  8, 46, 16,  4, 44,  5,  0,  9])
tensor([20,  1, 43,  8, 25, 27,  5, 32,  9, 21, 30,  1, 38, 38, 11, 40])
tensor([11, 26, 16, 36, 44, 30, 26, 17, 19, 29, 29,  3, 24, 24, 47,  7])
tensor([10, 30, 25, 17,  0, 34, 13, 34, 36, 27,  2,  5, 28, 19, 22, 33])
tensor([15, 35, 22, 29, 36, 12, 16, 37, 10, 25,  1, 26, 13, 32, 36, 15])
tensor([45, 24, 21, 21,  9,  3, 28, 14, 37, 20, 41, 29,  0,  7, 20, 46])
tensor([46, 24, 31,  2, 41, 49, 19, 31, 48, 39, 32, 33, 14, 45, 15, 36])
tensor([12, 27, 12, 11, 44, 38, 18, 19,  5, 10, 16,  4, 34, 19, 11, 11])
tensor([34,  0,  4, 15, 13,  7, 11, 22, 18, 39, 17, 27, 34, 43, 14, 10])
tensor([43, 36,  4, 11, 48, 30, 15, 21, 25,  1, 21, 21, 19, 45, 29, 45])
tensor([13, 29,  7, 30, 16, 20, 49, 20, 48, 23, 44, 32, 16, 35, 35, 45])
tensor([ 6, 35, 33, 20, 15, 41, 16, 32, 48, 44, 41,  2, 27, 10, 38, 13])
tensor([39,  8,  9, 36, 48,  2, 18, 24, 33, 42,  3,

tensor([13, 42, 38, 15, 37, 32, 36, 17, 22, 21,  4,  7,  5, 29, 40,  5])
tensor([ 8,  8, 27, 39,  2, 22, 30, 39, 41, 36,  1, 12, 19, 21, 45, 13])
tensor([34,  0, 32, 46, 25, 48, 47, 35, 32,  3,  2, 17, 45,  7, 49, 24])
tensor([ 5, 28, 27, 32, 17, 35,  2, 41, 14, 24,  2,  2, 22, 38, 43, 21])
tensor([ 9, 38, 19, 29, 16,  7, 41,  1, 35, 33, 33, 29, 38, 41, 21, 36])
tensor([30, 42, 44, 20, 43,  0,  5, 42,  4,  5,  5,  9, 14, 18, 38, 20])
tensor([48,  7, 30, 46, 16, 30,  5, 22, 44, 46, 22, 40, 18, 43, 45, 31])
tensor([38, 27, 31, 37, 34, 42, 24, 23, 48,  3, 15, 15, 22,  5, 22, 30])
tensor([45, 42, 30, 27, 14,  1, 16,  7, 17, 38, 35, 34, 15, 10,  2, 11])
tensor([35,  9,  1, 40, 18, 31, 28,  2, 40,  7, 18, 34, 17, 45, 24,  5])
tensor([41,  3, 10, 10,  9, 36, 18, 25,  9,  0,  5, 45, 22, 41, 18, 40])
tensor([26, 32, 32,  7, 24, 35, 30, 20, 13, 46,  2, 13,  5, 22, 16, 39])
tensor([46, 16, 42, 24, 40, 37, 24, 13, 26, 31, 41, 34, 48, 39, 27, 40])
tensor([16,  0, 18, 45, 48, 39,  4, 13, 32, 43, 16,

In [6]:
#feat,lbls = extract_features(model,train_data_loader)
test_feat,test_lbls = extract_features(model,test_data_loader)
lbls = flatten_list(lbls)
test_lbls  =flatten_list(test_lbls) # flatting the lbls

tensor([44, 19, 15, 37, 35, 44,  1, 20,  9, 47,  7, 24, 22, 42, 10, 28])
tensor([32, 17, 36,  0, 25, 27, 43, 17, 13, 15, 45, 12, 34, 29, 22,  6])
tensor([34, 26,  5, 41, 21, 13, 42, 23, 48, 11, 49, 32, 12, 24,  1, 44])
tensor([34, 25, 32, 23, 40, 10, 10, 16,  0, 10, 31, 38,  2, 27, 26, 13])
tensor([23, 34, 26, 32, 46, 43, 26, 30,  5,  3, 36, 44, 30, 19, 48, 20])
tensor([28, 11, 14, 37, 48, 49, 28, 37, 37,  2, 15, 37, 11, 37, 43, 40])
tensor([ 3, 17,  0, 27, 33, 19, 18, 11, 10,  5, 30,  0,  1, 18, 16, 18])
tensor([31, 39, 15, 35,  8, 42, 37, 34, 48, 37, 44, 38, 37,  6,  6, 49])
tensor([29, 40, 16, 43,  7, 19, 39, 16, 19, 25, 41,  5,  3, 17, 35, 29])
tensor([16, 24, 35, 44, 44, 33, 10, 40, 31, 36, 19, 24, 42, 14,  4, 23])
tensor([10, 43, 37, 39, 31, 13, 30, 38,  6, 28,  8, 15,  3, 17, 10,  7])
tensor([ 1, 14, 45,  5,  2, 14, 32, 45, 42,  3, 45, 20, 22, 36, 20, 35])
tensor([21, 13, 12, 34, 37, 19, 49, 43,  1, 48, 29, 34, 19, 32, 24, 20])
tensor([17, 38, 15,  0, 19, 16,  0, 14, 31,  1, 33,

tensor([10, 49, 41,  7, 26, 36, 18,  1,  8, 44,  8,  3,  3,  1])


In [7]:
len(lbls),len(test_lbls)

(4835, 1854)

In [8]:
feat.shape,len(lbls),test_feat.shape,len(test_lbls)

((4835, 1536), 4835, (1854, 1536), 1854)

In [13]:
# returns SVM accuracy
# accepts train and test feature set

from sklearn import metrics

def svm(X,Y,x_lbls,y_lbls):
    print('Train-Without FFT')
    #SVM
    svm = SVC(kernel='linear').fit(X,x_lbls)
    preds = svm.predict(Y)
    print("SVM Accuracy:",metrics.accuracy_score(y_lbls, preds))
    
svm(cat_feat_train,cat_feat_test,lbls,test_lbls)


NameError: name 'cat_feat_train' is not defined

In [9]:
from sklearn import metrics
print('Train-Without FFT')
#SVM
svm_preds = SVC(kernel='linear').fit(feat,lbls).predict(test_feat)
print("SVM Accuracy:",metrics.accuracy_score(test_lbls, svm_preds))


Train-Without FFT
SVM Accuracy: 0.5609492988133765


In [10]:
#save np network features and labels
# Use pickle for serialization or np.save for saving np objects. two options



i = 0 #video index 4797
j = 0 # neuron index 1000

def hstft(S1):
    S21, S22 = np.split(S1,2)  # 500+500
    S311, S312 = np.split(S21,2) # 500    
    S321, S322 = np.split(S22,2) # 500
    #S1.shape,S21.shape,S22.shape,S311.shape,S312.shape,S321.shape,S322.shape    
    return np.concatenate((S1,S21,S22,S311,S312,S321,S322))

def alpha(features):
    A = np.zeros((len(features),4608))
    for i,f in enumerate(features):
        h = scipy.fft.fft(f)    
        #print(f.shape,h.shape)
        #print(h.shape)
        h = hstft(h)
        #print(h.shape)
        A[i,:] = h.real
    return A

train_alpha = alpha(feat)
test_alpha = alpha(test_feat)
train_alpha.shape,test_alpha.shape

((4835, 4608), (1854, 4608))

In [11]:
X = train_alpha
Y = lbls
test_x = test_alpha
test_y = test_lbls
X.shape,Y.shape,test_x.shape,test_y.shape

((4835, 4608), (4835,), (1854, 4608), (1854,))

In [12]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#SVM
clf = SVC(kernel='linear').fit(X,Y)
preds = clf.predict(test_x)
print("SVM Accuracy:",metrics.accuracy_score(test_y, preds))

#KNN

knn_clf = KNeighborsClassifier(n_neighbors=3).fit(X,Y)
knn_preds = knn_clf.predict(test_x)
print("KNN Accuracy:",metrics.accuracy_score(test_y, knn_preds))

#Random Forest
rf_clf = RandomForestClassifier(n_estimators=100).fit(X,Y)
rf_preds = rf_clf.predict(test_x)
print("RF Accuracy:",metrics.accuracy_score(test_y, rf_preds))


SVM Accuracy: 0.5285868392664509
KNN Accuracy: 0.38349514563106796
RF Accuracy: 0.48327939590075514
